In [7]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
# Change to current dataset
import os
os.chdir("/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Cleaned_Code")

In [17]:
# Module Imports
import cv2
import glob
import math
import keras
import pickle
import random
import imutils
import numpy as np
import tensorflow as tf
from datetime import datetime
from matplotlib import pyplot as plt

from os import path
from skimage.io import imsave
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from scipy import ndimage # For rotation task or
from google.colab.patches import cv2_imshow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, AveragePooling2D, Reshape, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.layers import Activation, Input, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [10]:
# Check to see if GPU is being used
tf.test.gpu_device_name()

'/device:GPU:0'

In [11]:
# Dataset locations - TO BE MODIFIED FOR NEW DATASET
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.tif']
def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            # print("filename:", filename)
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

# Dataset location
root_dir = '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/UCMerced_LandUse/Images/Splits_3/'
train_filenames = sorted(get_file_list(root_dir + "train"))
val_filenames = sorted(get_file_list(root_dir + "val"))

In [22]:
dims = (448, 448, 3)
bool_generate=False
batch_size = 32

# Network Architecture - TO BE MODIFIED FOR NEW MODELS
def createModel():
    model = Sequential(name="encoder")
    model.add(Input(shape=dims))
    
    # Encoder
    model.add(Conv2D(32, (3, 3), activation="relu", padding="same", strides=2))
    model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
    model.add(Conv2D(64, (3, 3), activation="relu", padding="same", strides=2))
    model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
    model.add(Conv2D(128, (3, 3), activation="relu", padding="same", strides=2))
    model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
    model.add(Conv2D(256, (3, 3), activation="relu", padding="same",strides=3))
    model.add(Conv2D(15, (3, 3), activation="relu", padding="same"))
    model.add(Conv2D(5,  (3, 3), activation="relu", padding="same"))
    
    # Decoder
    model.add(Conv2DTranspose(128, (3, 3), activation="relu", padding="same"))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(64, (3, 3), activation="relu", padding="same"))
    model.add(Conv2DTranspose(64, (3, 3), activation="relu", padding="same"))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(32, (3, 3), activation="relu", padding="same"))
    model.add(Conv2D(3, (3, 3), activation="tanh", padding="same"))
    model.add(UpSampling2D((2, 2)))

    return model

In [23]:
from keras import backend as K
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)


model = createModel()
model.compile(optimizer='adam',loss=[tf.keras.losses.BinaryCrossentropy(),f1_loss],loss_weights=[0.5, 0.5],metrics=['accuracy',f1,tf.keras.metrics.BinaryCrossentropy()])
callback_earlystop=EarlyStopping(monitor='loss',patience=5)

checkpoint_filepath = 'Models/checkpoint/model2_{epoch:04d}.h5'
callback_checkpoint = ModelCheckpoint(
     filepath=checkpoint_filepath,
     save_weights_only=False,
     period=1)

model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 19, 19, 256)       5900

In [24]:
train_dataGenerator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   channel_shift_range=0.1)

valid_dataGenerator=ImageDataGenerator(preprocessing_function=preprocess_input)


batch_size = 32
trainGenerator = train_dataGenerator.flow_from_directory(
        root_dir+'/train/',
        target_size=(224, 224),
        batch_size= batch_size,
        class_mode= None, 
        shuffle = False)

testGenerator=valid_dataGenerator.flow_from_directory(
        root_dir+'/val/',
        target_size=(224, 224),
        batch_size= batch_size,
        class_mode= None, 
        shuffle = False)

FileNotFoundError: ignored